### 0419 :: 1. 마리아 DB 연결

In [8]:
# 데이저 저장
import pymysql as mysql

conn = mysql.connect(host = 'localhost', user='root', password='g901558',
            db='test', charset='utf8')
cur = conn.cursor()
sql = "insert into muser values (5, 'user8', 'pw2345','박자동', '040-4544-4444')"
cur.execute(sql)
conn.commit()
conn.close()

In [9]:
# 데이블 생성
conn = mysql.connect(host = 'localhost', user='root', password='g901558',
            db='test', charset='utf8')
cur = conn.cursor()
sql = 'create table if not exists uinfo(id int, username char(12), email char(25), birthdayyear int(4))'
cur.execute(sql)
conn.commit()
conn.close()

In [10]:
# 데이터 조회
conn = mysql.connect(host = 'localhost', user='root', password='g901558',
            db='test', charset='utf8')
cur = conn.cursor()
sql = 'select * from uinfo'
cur.execute(sql)
result = cur.fetchall()
result

((2, '홍길동', 'bedro4@naver.com', 2001), (3, '박자동', 'aaa123@sa.kr', 1998))

In [20]:
# id pw 받아서 조회한 후 보여주기
conn = mysql.connect(host = 'localhost', user='root', password='g901558',
            db='test', charset='utf8')
cur = conn.cursor()

user_id = input('아이디를 입력해주세요 :')
user_pw = input('패스워드를 입력해주세요: ')

sql = 'select id from muser where id="' + user_id + '"'
cur.execute(sql)
result_id = cur.fetchall()

# 없는 값 조회하면 아무것도 안나오므로 값이 있는지 부터 확인~!

if len(result_id) >= 1:
    # 데이터가 어떤 형식으로  return 되는지 꼭 확인!
    if user_id == result_id[0][0] :
        sql = 'select pw from muser where id="' + user_id + '"'
        cur.execute(sql)
        result_pw = cur.fetchall()
        
        if user_pw == result_pw[0][0] :
            sql = 'select * from muser where id="' + user_id + '"'
            cur.execute(sql)
            result_all = cur.fetchall()
            print(f'{user_id}님의 정보는 {result_all}입니다')
        
        else :
            print('PW가 틀렸습니다')
            
#     else :
#         print(result_id)
    
else :
    print('미등록 ID입니다')
    
conn.commit()
conn.close()

아이디를 입력해주세요 :user2
패스워드를 입력해주세요: user7890
user2님의 정보는 ((2, 'user2', 'user7890', '홍길동', ''),)입니다


### 2. CSV 데이터 가져와 DB에 저장하기

In [21]:
import pandas as pd
import matplotlib.pyplot as plt
import time
import sys
import pymysql as mysql

In [25]:
# excel은 sheet_name 입력 필요함!
df = pd.read_excel('회원.xlsx', sheet_name='기본정보')
display(df.head())
df.shape

,관리번호,회원구분,이름,주민등록번호,직장명,생년월일,Tel,Hp,가입일
0,05-001,정회원,주민석,770202-2978978,교육과학기술부,1977-02-02,043-2745-0034,011-452-8597,2005-03-07
1,05-002,정회원,김은아,610803-2132285,보건복지가족부,1961-06-23,02-131-0067,010-6565-6987,2005-03-07
2,05-003,정회원,박출하,651206-2948256,교육과학기술부,1965-12-06,043-226-0081,010-2365-4587,2005-03-07
3,06-001,정회원,노은영,631027-2689336,보건복지가족부,1963-04-12,043-457-0072,016-252-9754,2006-04-07
4,06-002,정회원,강은광,580919-2307360,행정안전부,1957-11-03,02-302-0096,019-336-9564,2006-06-12


(11, 9)

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   관리번호    11 non-null     object        
 1   회원구분    11 non-null     object        
 2   이름      11 non-null     object        
 3   주민등록번호  11 non-null     object        
 4   직장명     11 non-null     object        
 5   생년월일    11 non-null     datetime64[ns]
 6   Tel     11 non-null     object        
 7   Hp      11 non-null     object        
 8   가입일     11 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(7)
memory usage: 920.0+ bytes


In [30]:
df.loc[0, '생년월일'], df.loc[0, '가입일']

(Timestamp('1977-02-02 00:00:00'), Timestamp('2005-03-07 00:00:00'))

In [54]:
connect = mysql.connect(host='localhost', password='g901558', user='root'
                       , db = 'test', charset='utf8', autocommit=True)
cursor = connect.cursor()

for i in df.index:
    sql = 'insert into member values(%s, %s, %s, %s, %s, %s, %s, %s, %s)'
    cursor.execute(sql, (df.iloc[i, 0], df.iloc[i, 1], df.iloc[i, 2], df.iloc[i, 3],
                        df.iloc[i, 4], str(df.iloc[i, 5])[:10], df.iloc[i, 6], df.iloc[i, 7],
                        str(df.iloc[i, 8])[:10]))
    
connect.commit()
connect.close()

In [55]:
df1 = pd.read_excel('./회원.xlsx', sheet_name = '상세정보')
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   관리번호    8 non-null      object
 1   결혼여부    8 non-null      object
 2   운전면허증   8 non-null      object
 3   취미      8 non-null      object
 4   특기(소질)  8 non-null      object
 5   관심분야    8 non-null      object
 6   본적      8 non-null      object
 7   주민등록상   8 non-null      object
 8   비고      3 non-null      object
dtypes: object(9)
memory usage: 704.0+ bytes


In [59]:
df1.head()

,관리번호,결혼여부,운전면허증,취미,특기(소질),관심분야,본적,주민등록상,비고
0,05-001,미혼,취득,인터넷,엑셀,게임,충북,경기,회장
1,05-002,기혼,미취득,등산,파워포인트,여행,서울,서울,NaN
2,06-002,미혼,취득,운동,엑셀,서적,서울,강원,부회장
3,07-001,기혼,취득,낚시,파워포인트,여행,강원,경기,NaN
4,07-002,미혼,미취득,인터넷,파워포인트,게임,경기,강원,NaN


In [62]:
df1= df1.fillna('')
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   관리번호    8 non-null      object
 1   결혼여부    8 non-null      object
 2   운전면허증   8 non-null      object
 3   취미      8 non-null      object
 4   특기(소질)  8 non-null      object
 5   관심분야    8 non-null      object
 6   본적      8 non-null      object
 7   주민등록상   8 non-null      object
 8   비고      8 non-null      object
dtypes: object(9)
memory usage: 704.0+ bytes


In [63]:
connect = mysql.connect(host='localhost', password='g901558', user='root'
                       , db = 'test', charset='utf8', autocommit=True)
cursor = connect.cursor()

sql_list = []

for i in df1.index:
    sql_list.append((df1.iloc[i,0], df1.iloc[i,1], df1.iloc[i,4], df1.iloc[i,5],
                     df1.iloc[i,6], df1.iloc[i,8]))
sql = 'insert into meminfo values(%s, %s, %s, %s, %s, %s)'
# execute는 튜플 형식으로 꼭 넣어줘야 함!
cursor.executemany(sql, tuple(sql_list))
connect.commit()
connect.close()

In [64]:
sql_list

[('05-001', '미혼', '엑셀', '게임', '충북', '회장'),
 ('05-002', '기혼', '파워포인트', '여행', '서울', ''),
 ('06-002', '미혼', '엑셀', '서적', '서울', '부회장'),
 ('07-001', '기혼', '파워포인트', '여행', '강원', ''),
 ('07-002', '미혼', '파워포인트', '게임', '경기', ''),
 ('07-004', '기혼', '한글', '서적', '서울', '총무'),
 ('08-001', '미혼', '한글', '서적', '경기', ''),
 ('08-002', '기혼', '파워포인트', '여행', '강원', '')]

### 0420: 네티즌 평점 크롤

In [100]:
import requests
from bs4 import BeautifulSoup as bs
import time
import pandas as pd
import pymysql as mysql
from tqdm import tqdm

1. 네이버 평점 크롤링 해서 데이터 가져와서 저장
(영화제목, 평점, 본문, 날짜)
2. 마리아 DB에 연결해서 저장

In [81]:
tr_list[0].find('td', class_='title').find('a').text

'공기살인'

In [82]:
tr_list[0].find('td', class_='title').find('em').text

'10'

In [89]:
tr_list[0].find('td', class_='title').text.split('\n')[5]

'사실을 최대한 담으면서도 더 많은 사람들이 보게 하기 위해서 스토리도 신경 쓴 흔적이 많이 보였습니다. '

In [95]:
tr_list[0].find_all('td', class_='num')[-1].text[-8:]

'22.04.24'

In [101]:
timeS=time.time()
mov_list=[]

for n in tqdm(range(1, 101)):
    url = 'https://movie.naver.com/movie/point/af/list.naver?&page=' + str(n)
    html = requests.get(url)
    soup = bs(html.text, 'html.parser')
    tbody = soup.find('tbody')
    tr_list = tbody.find_all('tr')
    
    for i in tr_list:
        ti = i.find('td', class_='title').find('a').text
        sc = i.find('td', class_='title').find('em').text
        re = i.find('td', class_='title').text.split('\n')[5]
        da = i.find_all('td', class_='num')[-1].text[-8:]
        mov_list.append((ti, sc, re, da))
timeE = time.time()
print(f'쇼요시간 : {timeE - timeS}')
        
df = pd.DataFrame(mov_list, columns=['영화제목', '평점', '리뷰', '날짜'])
df

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:13<00:00,  7.30it/s]

쇼요시간 : -13.740959644317627


,영화제목,평점,리뷰,날짜
0,신비한 동물들과 덤블도어의 비밀,7,,22.04.24
1,수퍼 소닉2,8,,22.04.24
2,신비한 동물들과 덤블도어의 비밀,10,처음에는 지루했는데 큰 기대없이 보면 영상미가 있었어요~~,22.04.24
3,1917,9,극장에서 이 영화를 볼 수 있어서 너무 행복했다!!!,22.04.24
4,앵커,9,진부하지 않았음. 삐걱거리지 않은 부모-자녀 관계가 어디 있으랴? 그 균열 쪽으로 ...,22.04.24
...,...,...,...,...
995,블랙라이트,6,너무 늙으셨나? 나도..,22.04.23
996,이상한 나라의 수학자,7,한지우 학생이 무공을 연마하듯 비밀과외를 받고 통쾌하게 금수저 학생들 보다 성적이 ...,22.04.23
997,브로큰 플라워,7,"옛사랑을찾아떠나는여행, 내인생을되돌아보라",22.04.23
998,앵커,2,개무서움 진짜 ㅠ 스릴러아니고 공포아니냐구요,22.04.23


In [103]:
df.isna().sum()

영화제목    0
평점      0
리뷰      0
날짜      0
dtype: int64

In [107]:
df1 = df[df['리뷰'] != '']
df1

,영화제목,평점,리뷰,날짜
2,신비한 동물들과 덤블도어의 비밀,10,처음에는 지루했는데 큰 기대없이 보면 영상미가 있었어요~~,22.04.24
3,1917,9,극장에서 이 영화를 볼 수 있어서 너무 행복했다!!!,22.04.24
4,앵커,9,진부하지 않았음. 삐걱거리지 않은 부모-자녀 관계가 어디 있으랴? 그 균열 쪽으로 ...,22.04.24
5,소설가의 영화,9,함부로 판단하지 않는다면 노골적으로 바라보는 시선일지라도 불편하지 않을 수 있다,22.04.24
6,공기살인,10,너무 유명한 일이라 대략적인 사건들을 알고있었지만 피해자분들의 상황을 배우분들의 연...,22.04.24
...,...,...,...,...
995,블랙라이트,6,너무 늙으셨나? 나도..,22.04.23
996,이상한 나라의 수학자,7,한지우 학생이 무공을 연마하듯 비밀과외를 받고 통쾌하게 금수저 학생들 보다 성적이 ...,22.04.23
997,브로큰 플라워,7,"옛사랑을찾아떠나는여행, 내인생을되돌아보라",22.04.23
998,앵커,2,개무서움 진짜 ㅠ 스릴러아니고 공포아니냐구요,22.04.23


In [110]:
df1.dtypes

영화제목    object
평점      object
리뷰      object
날짜      object
dtype: object

In [111]:
mov_list

[('신비한 동물들과 덤블도어의 비밀', '7', '', '22.04.24'),
 ('수퍼 소닉2', '8', '', '22.04.24'),
 ('신비한 동물들과 덤블도어의 비밀', '10', '처음에는 지루했는데 큰 기대없이 보면 영상미가 있었어요~~ ', '22.04.24'),
 ('1917', '9', '극장에서 이 영화를 볼 수 있어서 너무 행복했다!!! ', '22.04.24'),
 ('앵커',
  '9',
  '진부하지 않았음. 삐걱거리지 않은 부모-자녀 관계가 어디 있으랴? 그 균열 쪽으로 100배 줌인한 듯한 이야기. 이 영화 보고 저녁밥을 체했어요. ',
  '22.04.24'),
 ('소설가의 영화', '9', '함부로 판단하지 않는다면 노골적으로 바라보는 시선일지라도 불편하지 않을 수 있다 ', '22.04.24'),
 ('공기살인',
  '10',
  '너무 유명한 일이라 대략적인 사건들을 알고있었지만 피해자분들의 상황을 배우분들의 연기로 직접보면서 느끼니까 슬픔이 더욱 공감되고 이런 일들은 잊지 않고 기억하면서 똑같은 일이 반복되지 않게 해야겠다고 생각되는 영화입니다.. 이런 영화를 만들어주셔서 감사합니다. 우연하게 무대인사까지 볼 수 있는 영화를 보게되었는데 진심을 다해 연기해주신 배우분들까지 뵐 수 있어 더 뜻깊은 시간이었습니다. 감사합니다ㅜ ',
  '22.04.24'),
 ('내일의 기억', '10', '스릴러라서 재미로 보려고 했는데 알고 보니 정말 아름다운 영화네요. ', '22.04.24'),
 ('스페이스 잼', '5', '내용은 단순하고 좀 유치하긴 하지만 마이클 조던의 플레이와 만화의 조합은 신선. ', '22.04.24'),
 ('올드보이', '6', '유명한 영화라해서 봤는데 생각만큼은 그닥.참신하긴한데 잔인해서 거북 ', '22.04.24'),
 ('공기살인',
  '10',
  '기대안하고 봤는데 너무 재미있게 봤네요. 10년전 일이라 기억속에 살아져가던 사건을 다시한번 상기시킬수 있어 좋았고 앞으로는 이런 사건

In [131]:
# 이렇게 하면 튜블로 변환 가능! (intertuples 이용)
df1_tuple = list(df1.itertuples(index=False, name=None))

In [132]:
# 리뷰 없는 것들은 제거해서 넣었음!
connect = mysql.connect(host='localhost', password='g901558', user='root',
                       db='test', charset='utf8',autocommit=True)
cursor = connect.cursor()
sql = 'insert into movie2 values(%s, %s, %s, %s)'
cursor.executemany(sql, tuple(df1_tuple))
conncet.commit()
connect.close()

In [134]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 933 entries, 2 to 999
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   영화제목    933 non-null    object
 1   평점      933 non-null    object
 2   리뷰      933 non-null    object
 3   날짜      933 non-null    object
dtypes: object(4)
memory usage: 36.4+ KB


In [142]:
sent = pd.read_csv('C:/Users/Admin/Desktop/multi/0420/movie_review.txt')
sent

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
199995,4608761,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1
199996,5308387,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO,0
199997,9072549,그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다,0
199998,5802125,절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네,0


In [143]:
from textblob import TextBlob as tb
import nltk
from textblob.classifiers import NaiveBayesClassifier
import re

In [157]:
train_lst = []
for i in range(len(sent)):
    train_lst.append(
    (re.sub('[^가-힣a-zA-Z ]', '', str(sent.iloc[i, 1])),
    sent.iloc[i, -1])
    )
train_lst

[('아 더빙 진짜 짜증나네요 목소리', 0),
 ('흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나', 1),
 ('너무재밓었다그래서보는것을추천한다', 0),
 ('교도소 이야기구먼 솔직히 재미는 없다평점 조정', 0),
 ('사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다', 1),
 ('막 걸음마 뗀 세부터 초등학교 학년생인 살용영화별반개도 아까움', 0),
 ('원작의 긴장감을 제대로 살려내지못했다', 0),
 ('별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지정말 발로해도 그것보단 낫겟다 납치감금만반복반복이드라마는 가족도없다 연기못하는사람만모엿네',
  0),
 ('액션이 없는데도 재미 있는 몇안되는 영화', 1),
 ('왜케 평점이 낮은건데 꽤 볼만한데 헐리우드식 화려함에만 너무 길들여져 있나', 1),
 ('걍인피니트가짱이다진짜짱이다', 1),
 ('볼때마다 눈물나서 죽겠다년대의 향수자극허진호는 감성절제멜로의 달인이다', 1),
 ('울면서 손들고 횡단보도 건널때 뛰쳐나올뻔 이범수 연기 드럽게못해', 0),
 ('담백하고 깔끔해서 좋다 신문기사로만 보다 보면 자꾸 잊어버린다 그들도 사람이었다는 것을', 1),
 ('취향은 존중한다지만 진짜 내생에 극장에서 본 영화중 가장 노잼 노감동임 스토리도 어거지고 감동도 어거지', 0),
 ('냥 매번 긴장되고 재밋음', 1),
 ('참 사람들 웃긴게 바스코가 이기면 락스코라고 까고바비가 이기면 아이돌이라고 깐다그냥 까고싶어서 안달난것처럼 보인다', 1),
 ('굿바이 레닌 표절인것은 이해하는데 왜 뒤로 갈수록 재미없어지냐', 0),
 ('이건 정말 깨알 캐스팅과 질퍽하지않은 산뜻한 내용구성이 잘 버무러진 깨알일드', 1),
 ('약탈자를 위한 변명 이라 저놈들은 착한놈들 절대 아닌걸요', 1),
 ('나름 심오한 뜻도 있는 듯 그냥 학생이 선생과 놀아나는 영화는 절대 아님', 1),
 ('보면서 웃지 않는 건 불가능하다', 1),
 ('

- 머신 정확하게 돌려야 하면 코랩으로 해야하는데 시간이 오래 걸리므로... 간단하게 표본 적게 돌리겠습니다

In [178]:
df_pos = sent[sent['label'] == 1][['document', 'label']]
df_neg = sent[sent['label'] == 0][['document', 'label']]
df_train = pd.concat([df_pos.iloc[:750], 
                                 df_neg.iloc[:750]]).reset_index(drop=True)
df_test = pd.concat([df_pos.iloc[750:1000], 
                                 df_neg.iloc[750:1000]]).reset_index(drop=True)


In [179]:
len(df_pos), len(df_neg)

(100000, 100000)

In [180]:
train_tuple = list(df_train.itertuples(index=False, name=None))
test_tuple = list(df_test.itertuples(index=False, name=None))

In [181]:
pos_neg = NaiveBayesClassifier(train_tuple)
test_acc = pos_neg.accuracy(test_tuple)

print(f'정확도: {test_acc}')

정확도: 0.714


In [186]:
df1['pos_neg'] = [pos_neg.classify(df.iloc[i, 2]) for i in range(len(df1))]
df1.head()

C:\Users\Admin\AppData\Local\Temp\ipykernel_11500\3847959224.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['pos_neg'] = [pos_neg.classify(df.iloc[i, 2]) for i in range(len(df1))]


,영화제목,평점,리뷰,날짜,pos_neg
2,신비한 동물들과 덤블도어의 비밀,10,처음에는 지루했는데 큰 기대없이 보면 영상미가 있었어요~~,22.04.24,0
3,1917,9,극장에서 이 영화를 볼 수 있어서 너무 행복했다!!!,22.04.24,0
4,앵커,9,진부하지 않았음. 삐걱거리지 않은 부모-자녀 관계가 어디 있으랴? 그 균열 쪽으로 ...,22.04.24,1
5,소설가의 영화,9,함부로 판단하지 않는다면 노골적으로 바라보는 시선일지라도 불편하지 않을 수 있다,22.04.24,1
6,공기살인,10,너무 유명한 일이라 대략적인 사건들을 알고있었지만 피해자분들의 상황을 배우분들의 연...,22.04.24,1


In [187]:
df1['pos_neg'].value_counts()

1    488
0    445
Name: pos_neg, dtype: int64

In [192]:
df1.dtypes

영화제목       object
평점         object
리뷰         object
날짜         object
pos_neg     int64
dtype: object

In [196]:
df1 = df1.astype({'평점' : 'int64'})

In [189]:
def movie(nm):
    sco = df1[df1['영화제목'] == nm]['평점'].mean()
    pn = df1[df1['영화제목'] == nm]['pos_neg'].mean()
    return print(f'평점 평균은 {sco}, 머신 평균은 {pn}입니다')

In [188]:
df1['영화제목'].unique()

array(['신비한 동물들과 덤블도어의 비밀', '1917', '앵커', '소설가의 영화', '공기살인', '내일의 기억',
       '스페이스 잼', '올드보이', '야차', '버티고', '중경삼림', '루카', '지골로 인 뉴욕', '테넷',
       '뜨거운 피', '스파이더맨: 노 웨이 홈', '해어화', '4월은 너의 거짓말', '드라이', '밀양',
       '얼어버린 시간 속에서', '로스트 시티', '더 웨이 홈', '겟 스마트', '오아시스', '언차티드',
       '이상한 나라의 수학자', '특송', '스펜서', '대부 : 끝나지 않은 이야기', '나인스 라이프', '그레이트 월',
       '종말의 끝', '아라한 장풍대작전', '수퍼 소닉2', '1987', '더 배트맨', '앰뷸런스', '노팅 힐',
       '개를 훔치는 완벽한 방법', '외출', '프랙처드', 'B컷', '알리타: 배틀 엔젤', '007 카지노 로얄 ',
       '해리 포터와 마법사의 돌', '이레이저', '목소리의 형태', '천국에서 무덤까지', '비트', '컨빅션',
       '2046', '해무', '샹치와 텐 링즈의 전설', '더 수어사이드 스쿼드', '소년시절의 너', '깡철이',
       '세븐틴 파워 오브 러브 : 더 무비', '파워 오브 도그', '도리화가', '월드 인베이젼', '원티드',
       '사냥의 시간', '해리 포터와 아즈카반의 죄수', '댄싱 온 글래스', '언니', '쇼크 웨이브 2', '트루맛쇼',
       '문폴', '트레이닝 데이: 비열한 거리', '극장판 귀멸의 칼날: 무한열차편', '인셉션', '루시드 드림',
       '모비우스', '말임씨를 부탁해', '타짜 전설의 땁', '노바디 ', '빅쇼트', '롱 리브 더 킹: 목포 영웅',
       '기적', '공작', '신과함께-인과 연', '드라이브 마이 카', '몸 값', '첨밀밀', '글로리데이',
       '팅커 테일

In [197]:
movie('신비한 동물들과 덤블도어의 비밀')

평점 평균은 6.819444444444445, 머신 평균은 0.5694444444444444입니다
